# Memory Information

In [79]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")
# keras-team /
# keras-contrib 

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.57GB
Used: 643.19MB
Percentage: 3.7%


# GPU Information

In [80]:
! nvidia-smi

Tue Nov 10 21:10:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [81]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [82]:
# Label Mapping
labels_mapping = {'defamation':0,
 'fake':1,
 'hate':2,
 'non-hostile':3,
 'offensive':4}

In [83]:
def evaluation(y_true, y_pred):
  print("Fine Grained Accuracy = {}".format(accuracy_score(y_true, y_pred)))
  print("\n\nFine Grained Metrics\n")
  print(classification_report(y_true, y_pred))

In [84]:
val_file = 'val.csv'

In [85]:
val_data = pd.read_csv(val_file, header=0, index_col=0)
val_y = np.empty((0, 5))
for index, row in val_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  val_y = np.vstack((val_y, y))

In [86]:
y_pred_defamation = pd.read_csv('y_pred_defamation.csv', header=None)[0].to_list()
y_pred_fake = pd.read_csv('y_pred_fake.csv', header=None)[0].to_list()
y_pred_hate = pd.read_csv('y_pred_hate.csv', header=None)[0].to_list()
y_pred_non_hostile = pd.read_csv('y_pred_non_hostile.csv', header=None)[0].to_list()
y_pred_offensive = pd.read_csv('y_pred_offensive.csv', header=None)[0].to_list()

In [87]:
y_pred = np.array((y_pred_defamation, y_pred_fake, y_pred_hate, y_pred_non_hostile, y_pred_offensive), dtype=int)

In [88]:
y_pred = y_pred.transpose()

In [89]:
evaluation(val_y, y_pred)

Fine Grained Accuracy = 0.6855733662145499


Fine Grained Metrics

              precision    recall  f1-score   support

           0       0.34      0.30      0.32        77
           1       0.82      0.79      0.81       160
           2       0.49      0.47      0.48       110
           3       0.98      0.97      0.97       435
           4       0.58      0.56      0.57       103

   micro avg       0.79      0.77      0.78       885
   macro avg       0.64      0.62      0.63       885
weighted avg       0.79      0.77      0.78       885
 samples avg       0.77      0.80      0.77       885



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [94]:
np.savetxt("y_pred.csv", y_pred, delimiter=",")